# Landscape map - stage 3

This stage merges the raw longlist with the candidate data from the reference sources to create the new list.

It round-trips the data from the existing landscape list to ensure that any manual overrides are included.

In [1]:
import duckdb
import petl as etl
from pipeline_utils.reference.geo import la_code_lookup
from pipeline_utils.reference.onspd import normalise_postcode, postcode_lookup
from pipeline_utils.filesystem.paths import RAW_DATA, DATA

Load the spelling corrections we have inferred from the matching stage.

In [2]:
corrections = etl.fromcsv(RAW_DATA / 'landscape-map-corrections.csv').lookupone('organisation', 'match')

Load the untagged longlist from the raw directory and perform the following operations:

1. Convert numeric data to numbers
2. Correct the spellings of the organisational data
3. Augment with local authority data

In [3]:
raw = etl.fromcsv(
    RAW_DATA / 'landscape-longlist-raw.csv'
).convertnumbers(
).convert(
    'organisation', lambda x: corrections.get(x, x)
).convert(
    'Local authority', la_code_lookup
).unpackdict(
    'Local authority'
).rename({
    'LAD24CD': 'funding_geo_code',
    'LAD24NM': 'funding_geo_name',
}).cache()

raw

organisation,Source,Number,funding_geo_code,funding_geo_name
4M Puppets,Project Grant,1,E08000021,Newcastle upon Tyne
Abdulrahman Abu - Zayd,Project Grant,1,E08000037,Gateshead
Action for Children,Project Grant,1,E06000057,Northumberland
Adam Phillips,Project Grant,3,E08000037,Gateshead
Adam Shield,Project Grant,1,E08000021,Newcastle upon Tyne


Get the list of sources in the longlist. We'll use this to update the values in the columns later on.

In [4]:
sources = tuple(raw.cut('Source').distinct().values('Source'))
sources

('IPSO', 'NPO', 'Project Grant')

Recast the longlist to convert the Source column into a column per entry, and convert any non-None values into True

In [5]:
wide_table = raw.recast(
    variablefield="Source",
    valuefield="Number"
).convert(
    sources,
    lambda x: True if x is not None else None
)
wide_table

organisation,funding_geo_code,funding_geo_name,IPSO,NPO,Project Grant
4M Puppets,E08000021,Newcastle upon Tyne,None,None,True
Abdulrahman Abu - Zayd,E08000037,Gateshead,None,None,True
Action for Children,E06000057,Northumberland,None,None,True
Adam Phillips,E08000037,Gateshead,None,None,True
Adam Shield,E08000021,Newcastle upon Tyne,None,None,True


At this point we will also add in new data from the result of stage 2.

1. `location` Manually set locations 
2. `companies` Company data from Companies house (direct and fuzzy matched)

In [6]:
location = etl.fromcsv(RAW_DATA / 'landscape-locations.csv').lookupone('organisation', ['latitude', 'longitude'])

Direct and fuzzy data is loaded from the database.

In [7]:
db = duckdb.connect(RAW_DATA / 'company-data.db', read_only=True)
db.query('''
         CREATE OR REPLACE TEMP TABLE tDirect AS
                SELECT *, 'direct' AS type, 100 as score
                FROM read_csv('../raw/landscape-map-company-direct.csv');
         CREATE OR REPLACE TEMP TABLE tFuzzy AS
                SELECT *
                FROM read_csv('../raw/landscape-map-company-fuzzy.csv');
         CREATE OR REPLACE TEMP TABLE tAllMatches AS
                SELECT organisation, company_number, type, score FROM tDirect
                UNION ALL
                SELECT match as organisation, CompanyNumber as company_number, type, score FROM tFuzzy;
         CREATE OR REPLACE TEMP TABLE tCharities AS
                SELECT *
                FROM read_csv('../raw/landscape-map-charity-direct.csv');
         ''')


In [8]:
companies = etl.fromdataframe(
    db.query('''
             SELECT m.organisation as organisation,
                c.CompanyName as registered_name,
                c.CompanyNumber as company_number,
                m.type,
                m.score,
                "URI" as uri,
                "RegAddress.PostTown" as post_town,
                "RegAddress.PostCode" as postcode,
                CompanyCategory as company_category,
                CompanyStatus as company_status,
                [x for x in [
                        "SICCode.SicText_1",
                        "SICCode.SicText_2",
                        "SICCode.SicText_3",
                        "SICCode.SicText_4"
                ] if x is not NULL] as sic_code,
                IncorporationDate as incorporation_date,
                DissolutionDate as dissolution_date,
                "Accounts.AccountCategory" as accounts_category
             FROM tAllMatches m
             JOIN CompanyData c
             ON m.company_number == c.CompanyNumber;
             ''').df())

In [9]:
charities = etl.fromdataframe(db.query('''
                     SELECT l.organisation, c.* FROM Charities c
                     JOIN tCharities l
                     ON 
                     c.registered_charity_number == l.registered_charity_number
                     ''').df())

In [10]:
db.close()

In [11]:
company_data = companies.dictlookupone('organisation')
charity_data = charities.dictlookupone('organisation')

Create the new landscape table

In [12]:
landscape = (
    wide_table
    .addfield('location', lambda r: location.get(r.organisation, ()))
    .unpack('location', newfields=['latitude', 'longitude'])
    .addfield('company_data', lambda r: company_data.get(r.organisation, {}))
    .unpackdict('company_data', keys=[
        'company_category',
        'accounts_category',
        'company_number',
        'company_status',
        'dissolution_date',
        'incorporation_date',
        'post_town',
        'postcode',
        'sic_code',
        'uri',
        'type', 'score'
    ])
    .convert('postcode', normalise_postcode)
    .addfield('charity_data', lambda r: charity_data.get(r.organisation, {}))
    .unpackdict('charity_data', keys=[
        'charity_name',
        'registered_charity_number',
        'charity_company_registration_number',
        'charity_contact_postcode',
        'charity_contact_web',
        'latest_expenditure',
        'latest_income',
    ])
    .convert('postcode', lambda x: postcode_lookup.get(x, { 'pcds': x }))
    .unpackdict('postcode', keys=['pcds', 'lat', 'long', 'oslaua'])
    .convert('latitude', lambda x, r: r['lat'], pass_row=True, where=lambda r: r['latitude'] == None and r['lat'] != None)
    .convert('longitude', lambda x, r: r['long'], pass_row=True, where=lambda r: r['longitude'] == None and r['long'] != None)
    .cutout('lat', 'long')
    .sort('organisation')
)
landscape.selectnotnone('charity_contact_postcode')

organisation,funding_geo_code,funding_geo_name,IPSO,NPO,Project Grant,latitude,longitude,company_category,accounts_category,company_number,company_status,dissolution_date,incorporation_date,post_town,sic_code,uri,type,score,charity_name,registered_charity_number,charity_company_registration_number,charity_contact_postcode,charity_contact_web,latest_expenditure,latest_income,pcds,oslaua
Curious Monkey Ltd,E08000021,Newcastle upon Tyne,None,None,True,54.974124,-1.647733,"PRI/LTD BY GUAR/NSC (Private, limited by guarantee, no share capital)",TOTAL EXEMPTION FULL,08550041,Active,None,2013-05-30 00:00:00,NEWCASTLE UPON TYNE,['90010 - Performing arts'],http://business.data.gov.uk/id/company/08550041,direct,100,CURIOUS MONKEY LTD,1167703,08550041,NE4 9PQ,www.curiousmonkeytheatre.com,224709,177333,NE4 9PQ,E08000021
Friends of Jesmond Library,E08000021,Newcastle upon Tyne,None,None,True,54.993462,-1.605726,"PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)",TOTAL EXEMPTION FULL,08430250,Active,None,2013-03-05 00:00:00,NEWCASTLE UPON TYNE,['91011 - Library activities'],http://business.data.gov.uk/id/company/08430250,direct,100,FRIENDS OF JESMOND LIBRARY,1151935,08430250,NE2 2DL,www.jesmondlibrary.org,58551,65592,NE2 2DL,E08000021
Liberdade community development trust,E08000021,Newcastle upon Tyne,None,None,True,55.011060,-1.630352,"PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)",GROUP,06548122,Active,None,2008-03-28 00:00:00,NEWCASTLE UPON TYNE,['90010 - Performing arts'],http://business.data.gov.uk/id/company/06548122,direct,100,LIBERDADE COMMUNITY DEVELOPMENT TRUST,1163955,06548122,NE3 3HD,www.liberdade.co.uk,771550,1448216,NE3 3HD,E08000021
Mortal Fools,E06000057,Northumberland,None,True,True,55.18376,-1.57168,"PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)",TOTAL EXEMPTION FULL,08102487,Active,None,2012-06-12 00:00:00,ASHINGTON,['85520 - Cultural education' '85590 - Other education n.e.c.' '90010 - Performing arts'],http://business.data.gov.uk/id/company/08102487,direct,100,MORTAL FOOLS,1153400,08102487,NE63 9XQ,www.mortalfools.org.uk,492393,434654,NE63 9XQ,E06000057
Opera Sunderland,E08000024,Sunderland,None,None,True,None,None,"PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)",MICRO ENTITY,08902353,Active,None,2014-02-19 00:00:00,SUNDERLAND,['90010 - Performing arts' '90020 - Support activities to performing arts' '90030 - Artistic creation'],http://business.data.gov.uk/id/company/08902353,direct,100,OPERA SUNDERLAND,1157157,08902353,SR3 3NR,www.operasunderland.co.uk,565,544,SR3 3NR,None


Finally, write the CSV file

In [13]:
landscape.tocsv(DATA / 'culture_landscape.csv')